In [2]:
import pandas as pd
import os

# ==============================================================================
# 0. ĐỊNH NGHĨA CÁC ĐƯỜNG DẪN
# ==============================================================================
BASE_DIR = r"E:\project 1"

# Đường dẫn file input
match_logs_path = os.path.join(BASE_DIR, "data_18_25", "matchlog", "players_standard_matchlogs.csv")
club_info_path = os.path.join(BASE_DIR, "data_clean", "club.xlsx")
player_info_path = os.path.join(BASE_DIR, "data_clean", "player_info.xlsx")

# Đường dẫn file output
output_path = os.path.join(BASE_DIR, "data_clean", "Club_player.xlsx")

print("Đã thiết lập các đường dẫn file.")

# ==============================================================================
# BẮT ĐẦU XỬ LÝ
# ==============================================================================
try:
    # 1. ĐỌC CÁC FILE DỮ LIỆU
    print(f"Đang đọc file match logs từ: {match_logs_path}")
    match_logs_df = pd.read_csv(match_logs_path)

    print(f"Đang đọc file thông tin CLB từ: {club_info_path}")
    club_info_df = pd.read_excel(club_info_path)

    print(f"Đang đọc file thông tin cầu thủ từ: {player_info_path}")
    player_info_df = pd.read_excel(player_info_path)
    print("-" * 50)

    # 2. TRÍCH XUẤT CÁC CẶP (PLAYER, SQUAD) DUY NHẤT
    print("Trích xuất các cặp (Cầu thủ, CLB) duy nhất từ match logs...")
    # Lấy 2 cột cần thiết và loại bỏ các hàng trùng lặp hoàn toàn
    club_player_df = match_logs_df[['Player', 'Squad']].drop_duplicates().reset_index(drop=True)
    print(f"Tìm thấy {len(club_player_df)} cặp (Cầu thủ, CLB) duy nhất.")
    print("-" * 50)

    # 3. MERGE ĐỂ LẤY CLUB_ID
    print("Bắt đầu gán Club_ID...")
    # Giả sử cột tên CLB trong file club.xlsx là 'Squad' để khớp với file match logs
    # Nếu tên cột khác (ví dụ: 'Club Name'), hãy thay đổi 'Squad' trong dòng dưới
    final_df = pd.merge(
        club_player_df,
        club_info_df[['Squad', 'Club_ID']], # Chỉ lấy các cột cần thiết từ file club
        on='Squad',
        how='left' # Giữ lại tất cả các cặp từ match logs, kể cả khi không tìm thấy CLB
    )
    print("Gán Club_ID hoàn tất.")
    print("-" * 50)

    # 4. MERGE ĐỂ LẤY Player ID
    print("Bắt đầu gán Player ID...")
    # Khớp cột 'Player' từ bảng đang xử lý với cột 'Name' trong file player_info
    final_df = pd.merge(
        final_df,
        player_info_df[['Name', 'Player ID']], # Chỉ lấy các cột cần thiết từ file player
        left_on='Player',
        right_on='Name',
        how='left' # Giữ lại tất cả, kể cả khi không tìm thấy cầu thủ
    )
    print("Gán Player ID hoàn tất.")
    print("-" * 50)

    # 5. KIỂM TRA DỮ LIỆU BỊ THIẾU SAU KHI MERGE
    missing_clubs = final_df[final_df['Club_ID'].isnull()]
    missing_players = final_df[final_df['Player ID'].isnull()]

    if not missing_clubs.empty:
        print(f"CẢNH BÁO: Không tìm thấy Club_ID cho {len(missing_clubs)} CLB sau:")
        print(missing_clubs['Squad'].unique())
    
    if not missing_players.empty:
        print(f"CẢNH BÁO: Không tìm thấy Player ID cho {len(missing_players)} cầu thủ sau:")
        print(missing_players['Player'].unique())

    # Loại bỏ những hàng không tìm thấy ID để đảm bảo dữ liệu sạch
    final_df.dropna(subset=['Player ID', 'Club_ID'], inplace=True)
    print("Đã loại bỏ các hàng không có Player ID hoặc Club_ID.")
    print("-" * 50)

    # 6. TẠO CỘT CLUB_Player ID
    print("Tạo cột Club_Player ID...")
    # Đảm bảo các ID là kiểu chuỗi (string) trước khi ghép
    final_df['Player ID'] = final_df['Player ID'].astype(str)
    final_df['Club_ID'] = final_df['Club_ID'].astype(str)
    
    final_df['Club_Player ID'] = final_df['Club_ID'] + '-' + final_df['Player ID']
    print("Tạo Club_Player ID hoàn tất.")
    print("-" * 50)

    # 7. CHUẨN BỊ BẢNG CUỐI CÙNG VÀ LƯU FILE
    # Chọn và sắp xếp lại các cột theo yêu cầu
    output_df = final_df[['Club_Player ID', 'Player ID', 'Club_ID']]

    # Lưu ra file Excel
    output_df.to_excel(output_path, index=False)
    print(f"THÀNH CÔNG! Đã lưu {len(output_df)} dòng vào file:")
    print(output_path)
    print("\nXem trước 5 dòng đầu của file kết quả:")
    print(output_df.head())

except FileNotFoundError as e:
    print(f"LỖI: Không tìm thấy file. Vui lòng kiểm tra lại đường dẫn.")
    print(f"Chi tiết lỗi: {e}")
except KeyError as e:
    print(f"LỖI: Tên cột không đúng. Vui lòng kiểm tra lại tên các cột trong file Excel/CSV.")
    print(f"Cột bị thiếu: {e}")
except Exception as e:
    print(f"Đã xảy ra một lỗi không mong muốn: {e}")

Đã thiết lập các đường dẫn file.
Đang đọc file match logs từ: E:\project 1\data_18_25\matchlog\players_standard_matchlogs.csv
Đang đọc file thông tin CLB từ: E:\project 1\data_clean\club.xlsx
Đang đọc file thông tin cầu thủ từ: E:\project 1\data_clean\player_info.xlsx
--------------------------------------------------
Trích xuất các cặp (Cầu thủ, CLB) duy nhất từ match logs...
Tìm thấy 1955 cặp (Cầu thủ, CLB) duy nhất.
--------------------------------------------------
Bắt đầu gán Club_ID...
Gán Club_ID hoàn tất.
--------------------------------------------------
Bắt đầu gán Player ID...
Gán Player ID hoàn tất.
--------------------------------------------------
CẢNH BÁO: Không tìm thấy Club_ID cho 7 CLB sau:
['CSKA Moscow' 'Spartak Moscow' 'Dynamo Kyiv' 'Rostov' 'Shakhtar']
Đã loại bỏ các hàng không có Player ID hoặc Club_ID.
--------------------------------------------------
Tạo cột Club_Player ID...
Tạo Club_Player ID hoàn tất.
--------------------------------------------------
THÀNH

hàng tuần 

In [1]:
import pandas as pd
import os
from datetime import datetime

# ==============================================================================
# 0. ĐỊNH NGHĨA CÁC ĐƯỜNG DẪN
# ==============================================================================
BASE_DIR = r"E:\project 1"

# Đường dẫn file input (Cập nhật theo yêu cầu mới)
match_logs_path = os.path.join(BASE_DIR, "data_2025_2026", "matchlog_2526", "players_standard_matchlogs.csv")
club_info_path = os.path.join(BASE_DIR, "data_clean", "club.xlsx")
player_info_path = os.path.join(BASE_DIR, "data_clean", "player_info.xlsx")

# Đường dẫn file output
master_path = os.path.join(BASE_DIR, "data_clean", "Club_player.xlsx")
import_path = os.path.join(BASE_DIR, "data_2025_2026", "data_import", "Club_player_import.xlsx")
log_path = os.path.join(BASE_DIR, "log", "Club_player.txt")

# Đảm bảo thư mục log và import tồn tại
os.makedirs(os.path.dirname(import_path), exist_ok=True)
os.makedirs(os.path.dirname(log_path), exist_ok=True)

print("Đã thiết lập các đường dẫn file.")

# ==============================================================================
# BẮT ĐẦU XỬ LÝ
# ==============================================================================
try:
    # 1. ĐỌC CÁC FILE DỮ LIỆU
    print(f"Đang đọc file match logs mới: {match_logs_path}")
    match_logs_df = pd.read_csv(match_logs_path)
    club_info_df = pd.read_excel(club_info_path)
    player_info_df = pd.read_excel(player_info_path)

    # 2. TRÍCH XUẤT CÁC CẶP (PLAYER, SQUAD) DUY NHẤT TỪ FILE MỚI
    new_pairs = match_logs_df[['Player', 'Squad']].drop_duplicates().reset_index(drop=True)

    # 3. MERGE ĐỂ LẤY ID
    # Lấy Club_ID
    new_pairs = pd.merge(new_pairs, club_info_df[['Squad', 'Club_ID']], on='Squad', how='left')
    # Lấy Player ID
    new_pairs = pd.merge(new_pairs, player_info_df[['Name', 'Player ID']], left_on='Player', right_on='Name', how='left')

    # Loại bỏ hàng thiếu ID
    new_pairs.dropna(subset=['Player ID', 'Club_ID'], inplace=True)

    # Tạo Club_Player ID
    new_pairs['Player ID'] = new_pairs['Player ID'].astype(str).str.replace('.0', '', regex=False)
    new_pairs['Club_ID'] = new_pairs['Club_ID'].astype(str).str.replace('.0', '', regex=False)
    new_pairs['Club_Player ID'] = new_pairs['Club_ID'] + '-' + new_pairs['Player ID']

    # Chuẩn bị DataFrame kết quả xử lý xong
    processed_df = new_pairs[['Club_Player ID', 'Player ID', 'Club_ID']].drop_duplicates()

    # 4. KIỂM TRA TRÙNG LẶP VỚI FILE HIỆN TẠI (MASTER)
    if os.path.exists(master_path):
        existing_master_df = pd.read_excel(master_path)
        existing_master_df['Club_Player ID'] = existing_master_df['Club_Player ID'].astype(str)
        
        # Lọc ra những ID chưa có trong file master
        new_records = processed_df[~processed_df['Club_Player ID'].isin(existing_master_df['Club_Player ID'])]
    else:
        existing_master_df = pd.DataFrame(columns=['Club_Player ID', 'Player ID', 'Club_ID'])
        new_records = processed_df

    # 5. XỬ LÝ LƯU DỮ LIỆU NẾU CÓ BẢN GHI MỚI
    if not new_records.empty:
        # A. Cập nhật file Master (Append)
        updated_master_df = pd.concat([existing_master_df, new_records], ignore_index=True)
        updated_master_df.to_excel(master_path, index=False)
        print(f"Đã cập nhật {len(new_records)} dòng mới vào file Master.")

        # B. Lưu file Import (Chỉ chứa dòng mới)
        new_records.to_excel(import_path, index=False)
        print(f"Đã lưu file import tại: {import_path}")

        # C. Ghi Log
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        new_ids_list = ", ".join(new_records['Club_Player ID'].tolist())
        
        with open(log_path, "a", encoding="utf-8") as f:
            f.write(f"--- Cập nhật ngày: {now} ---\n")
            f.write(f"Số lượng ID mới: {len(new_records)}\n")
            f.write(f"Danh sách ID: {new_ids_list}\n")
            f.write("-" * 50 + "\n")
        
        print("Đã ghi log thành công.")
    else:
        print("Không có ID mới nào được tìm thấy. Không có gì để cập nhật.")

except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")
    import traceback
    traceback.print_exc()

Đã thiết lập các đường dẫn file.
Đang đọc file match logs mới: E:\project 1\data_2025_2026\matchlog_2526\players_standard_matchlogs.csv
Đã cập nhật 20 dòng mới vào file Master.
Đã lưu file import tại: E:\project 1\data_2025_2026\data_import\Club_player_import.xlsx
Đã ghi log thành công.
